<div style="align-items: center; justify-content: space-between;">
   
   <h2>Data Pipeline Telegram</h2>
   <b> Nome: <a href="https://www.linkedin.com/in/fernandohcarneiro/">Ederson Santos</a></b>
   <br><br>
   <img src="https://raw.githubusercontent.com/edersonss1987/AutomacaoTelegram/57fd76228b42a7e5c556113975d0ad5be782693e/imagens/Fonte%20dos%20dados.png"  align="center" alt="data-pipeline" width="auto">

</div>


## Sumário
#### 1. [**Introdução**](#intro)  
    1.1 Objetivo  
    1.2 O que é Pipeline?  
    1.3 Pipeline do projeto  
---
#### 2. [**Etapa transacional**](#sistrans)  
    2.1 Ingestão de dados  
    2.2 O que é uma API?  
    2.3 Telegram Botfather  
    2.4 Webhook  
    2.5 AWS API Gateway  
---
#### 3. [**Etapa analítico**](#sisanal)  
    3.1 O que é ETL (Extraction, Transformation, Loading)?  
    3.2 Extração    
        3.2.1 AWS Lambda  
        3.2.2 AWS S3
        3.2.3 Extração Telegram   
    3.3 Transformação
    3.4 Carregamento    
---
#### 4. [**Apresentação**](#apres)  
    4.1 AWS Athena  
    4.2 Análise de Dados
#### 5. [**Conclusão**](#conclu)  

<a id='intro'></a>
## 1. Introdução

### 1.1 Objetivo
O objetivo deste projeto é apresentar o processo de extração de dados do Telegram, em sua forma original em um formaro semi-estruturado(.JSON),transferir esses dados para um ***Datalake***, realizar o processamento dos dados de forma automatizada e então fazer a análise dos dados tratados, usando linguagem SQL. Esses dados fornecem a base para extrair informações valiosas, abrindo possibilidades para aprimorar serviços e explorar oportunidades.

### 1.2 O que é Pipeline?
A palavra “pipeline” em inglês é frequentemente usada em português com o mesmo significado, especialmente em argumentos técnicos ou de negócios. No entanto ele é muito utilizado em um contexto de processamento de dados ou de projetos, pode ser traduzida como “fluxo de trabalho” ou “processo”.

Um Pipeline de Dados é uma série de etapas de processamento de dados, onde a saída de uma etapa é a entrada para a próxima. Essas etapas podem incluir coleta, limpeza, transformação, modelagem e visualização de dados.

### 1.3 Pipeline do projeto
O pipeline de dados deste projeto inicia com a ingestão dos dados de um aplicavo de mensagens, bastante conhecido *TELEGRAM*, as trocas de mensagens são coletadas via *WEBHOOK*, conectada a *API* da *Amazon Web Services (AWS)*. Na plataforma da *AWS*, os dados são recebidos por uma função *Lambda*, que os organiza por dias no *AWS S3*. Diariamente, um AWS *Event Bridge* aciona um processo em lote no serviço *Lambda*, que transforma os dados brutos, extrai apenas as informações relevantes (data da mensagem,user_id, nome do contato,texto digitado e tipo de chat(grupo)) e os armazena de maneira organizada no *AWS S3*. No processo de visualização, tabelas criadas a partir dos arquivos *Parquet* gerados no passo anterior possibilitam a realização de análises variadas usando a linguagem *SQL*.

![Pipeline Telegram](https://raw.githubusercontent.com/edersonss1987/AutomacaoTelegram/57fd76228b42a7e5c556113975d0ad5be782693e/imagens/Fonte%20dos%20dados.png)

<a id='sistrans'></a>
## 2. Sistema Transacional

Um sistema transacional é uma estrutura tecnológica e de dados que suporta as transações diárias de uma fonte de dados. Ele é responsável por coletar, registrar e transmitir dados para uma cadeia de processos que tratarão os dados para posterior análise. Em resumo, os sistemas transacionais são projetados para ingerir dados criados diariamente e salvá-los em um banco de dados ou *DataLake*.



### 2.1 Ingestão de Dados
Neste projeto a ingestão dos dados consiste na captura de mensagens de texto enviadas pelas plataformas Whatsapp e Telegram em tempo real, fornecidas por meio de uma API.

#### 2.2 O que é uma API?
Uma API, ou Interface de Programação de Aplicações, é um componente de software dentro de um sistema que fornece um mecanismo para invocar uma tarefa em outro sistema. Ela serve como uma ponte que facilita a comunicação e integração entre sistemas ou componentes de software diversos, até de tipos diferentes.



#### 2.3 AWS API Gateway
O *API Gateway* é o primeiro serviço em nuvem na sequência do pipeline, faz parte dos serviços em nuvem da *Amazon Web Services (AWS)*. Este serviço centraliza o gerenciamento das APIs inclusive o recebimento, como no caso deste projeto, em que servirá como porta de entrada para o recebimento do *payload* advindo das plataformas de mensagens. Ele oferece recursos como autenticação, autorização, monitoramento e escalabilidade, simplificando o processo de construção e administração de APIs de forma eficiente na infraestrutura da AWS.



##### 2.3.1 Criação da API para Telegram
Na plataforma *Telegram*, foi criado uma nova API com protocolo REST com método POST, configurado com integração do tipo proxy com o serviço Lambda.

<img src="https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/API%20Telegram.png?raw=true">

#### 2.4 Webhook
Um Webhook é um método de comunicação entre sistemas na web que permite a um sistema notificar outro quando um evento específico ocorre. Ele funciona como uma “chamada de retorno” HTTP: um servidor envia uma solicitação HTTP para a URL de um webhook configurado em outro sistema quando o evento ocorre. Isso permite que o sistema receptor seja atualizado ou tome uma ação imediatamente após o evento. É amplamente utilizado para integrações e automações entre diferentes serviços web.


##### 2.4.1 Configuração do Telegram
O procedimento na plataforma *Telegram* foi preciso acionar o método `setWebhook`. Abaixo segue a captura da execução da requisição:
```python
requests.get(f'https://api.telegram.org/bot{token}/setWebhook?url={base_url}')
```

<img src= "https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/setWebhook.png?raw=true">

##### 2.5.1 Configuração do Telegram


Após configurado o *webhook* foi possível verificar seu funcionamento com o método `getWebhookInfo`:

<img src="https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/getWebhootInfo.png?raw=true">


<a id='sisanal'></a>
## 3. Sistema analítico
Esses sistemas apoiam a tomada de decisões, relatórios, consultas e análises. São projetados para lidar com consultas complexas em grandes volumes de dados vindos dos sistemas transacionais, organizam esses dados e os processam de maneira a criar insights úteis. Neste projeto o sistema compreende a retirada dos dados brutos (*raw*) do datalake, a transformação deles em informação e a análise em busca de padrões e insights.



```python
{
 "ok": true,
 "result": [
  {
   "update_id": 126178061,
   "message": {
    "message_id": 282,
    "from": {
     "id": 6310508671,
     "is_bot": false,
     "first_name": "Eder ControliD",
     "language_code": "pt-br"
    },
    "chat": {
     "id": -1002096546090,
     "title": "Eder_IDBot",
     "type": "supergroup"
    },
    "date": 1715387518,
    "text": "Mensagens capturadas pelo telegram"
   }
  },
  {
   "update_id": 126178062,
   "message": {
    "message_id": 283,
    "from": {
     "id": 5154776950,
     "is_bot": false,
     "first_name": "Eder",
     "language_code": "pt-br"
    },
    "chat": {
     "id": -1002096546090,
     "title": "Eder_IDBot",
     "type": "supergroup"
    },
    "date": 1715387622,
    "text": "Todas as mensagens est\u00e3o automaticamente sendo salvas no nosso DataLake"
   }
  }
 ]
}
```


### 3.1 O que é ETL ?
É o processo de extrair, limpar e organizar os dados de uma origem para serem carregados, ou armazenados, em um local específico. Essa etapa é crucial para assegurar que os dados estejam em uma forma apropriada e prontos para análise antes de serem empregados em relatórios, visualizações ou em outros procedimentos analíticos.


### 3.2 Extração
Envolve a extração, ou coleta, de dados brutos de diferentes fontes, como as APIs neste caso.

#### 3.2.1 AWS Lambda
O AWS Lambda é um serviço de computação que permite a execução de código sem a necessidade de gerenciar servidores, também chamado de *serverless*. Nela é possível criar funções sem se preocupar com o escalamento e infraestrutura subjacente. Neste projeto ela se encarregará de executar scripts na linguagem *Python* para manuseio e transformação dos dados.


#### 3.2.2 AWS S3
O *Amazon S3*, ou *Simple Storage Service*, é um serviço de armazenamento que permite armazenar e recuperar dados de maneira fácil e escalável. É amplamente utilizado para armazenar arquivos, fazer backup e hospedar sistemas online, proporcionando uma solução eficiente e confiável para necessidades de armazenamento na nuvem. Neste projeto, ele armazenará tanto os dados brutos servindo como *Datalake*, quanto os dados transformados e prontos para análise.



##### 3.2.3 Extração Telegram
Abaixo segue o script em Python usado para a extração dos dados do Telegram.

```python
import boto3


def lambda_handler(event: dict, context: dict) -> dict:

  '''
  Recebe uma mensagens do Telegram via AWS API Gateway, verifica no
  seu conteúdo se foi produzida em um determinado grupo e a escreve,
  em seu formato original JSON, em um bucket do AWS S3.
  '''

  # vars de ambiente

  BUCKET = os.environ['AWS_S3_BUCKET']
  TELEGRAM_CHAT_ID = int(os.environ['TELEGRAM_CHAT_ID'])

  # vars lógicas

  tzinfo = timezone(offset=timedelta(hours=-3))
  date = datetime.now(tzinfo).strftime('%Y-%m-%d')
  timestamp = datetime.now(tzinfo).strftime('%Y%m%d%H%M%S%f')

  filename = f'{timestamp}.json'

  # código principal

  client = boto3.client('s3')

  try:

    message = json.loads(event["body"])
    chat_id = message["message"]["chat"]["id"]

    if chat_id == TELEGRAM_CHAT_ID:

      with open(f"/tmp/{filename}", mode='w', encoding='utf8') as fp:
        json.dump(message, fp)

      client.upload_file(f'/tmp/{filename}', BUCKET, f'telegram/context_date={date}/{filename}')

  except Exception as exc:
      logging.error(msg=exc)
      return dict(statusCode="500")

  else:
      return dict(statusCode="200")

```

Note que em ambas as extrações os dados provenientes da *API* em formato *JSON* são direcionados para um endereço na *web* armazenado na variável `BUCKET`. Este é o nome do espaço de armazenamento do serviço *AWS S3* para onde estamos enviando os dados brutos. Na sequência será explanado a etapa seguinte, a de transformação dos dados extraídos.

Nesta etapa, os dados coletados são processados e transformados para atender aos requisitos do destino. Isso inclui limpeza, filtragem, agregação e qualquer manipulação necessária.

#### 3.3.1 Transformação Telegram (Lambda)
O script abaixo, de forma similar, recebe os dados brutos de um *bucket* e retira dele somente as informações de interesse, devolvendo o dado estruturado no formato de tabela *parquet*. A diferença entre os scripts da plataforma *Whatsapp* e *Telegram* é basicamente a estrutura em que o dado é recebido via *API*, sendo o do *Whatsapp* maior e com mais aninhamento entre itens.

```python

import os
import json
import logging
from datetime import datetime, timedelta, timezone

import boto3
import pyarrow as pa
import pyarrow.parquet as pq


def lambda_handler(event: dict, context: dict) -> bool:

  '''
  Diariamente é executado para compactar as diversas mensagensm, no formato
  JSON, do dia anterior, armazenadas no bucket de dados cru, em um único
  arquivo no formato PARQUET, armazenando-o no bucket de dados enriquecidos
  '''

  # vars de ambiente

  RAW_BUCKET = os.environ['AWS_S3_BUCKET']
  ENRICHED_BUCKET = os.environ['AWS_S3_ENRICHED']

  # vars lógicas

  tzinfo = timezone(offset=timedelta(hours=-3))
  date = (datetime.now(tzinfo) - timedelta(days=1)).strftime('%Y-%m-%d')
  timestamp = datetime.now(tzinfo).strftime('%Y%m%d%H%M%S%f')

  # código principal

  table = None
  client = boto3.client('s3')

  try:

      response = client.list_objects_v2(Bucket=RAW_BUCKET, Prefix=f'telegram/context_date={date}')

      for content in response['Contents']:

        key = content['Key']
        client.download_file(RAW_BUCKET, key, f"/tmp/{key.split('/')[-1]}")

        with open(f"/tmp/{key.split('/')[-1]}", mode='r', encoding='utf8') as fp:

          data = json.load(fp)
          data = data["message"]

        parsed_data = parse_data(data=data)
        iter_table = pa.Table.from_pydict(mapping=parsed_data)

        if table:

          table = pa.concat_tables([table, iter_table])

        else:

          table = iter_table
          iter_table = None

      pq.write_table(table=table, where=f'/tmp/{timestamp}.parquet')
      client.upload_file(f"/tmp/{timestamp}.parquet", ENRICHED_BUCKET, f"telegram/context_date={date}/{timestamp}.parquet")

      return True

  except Exception as exc:
      logging.error(msg=exc)
      return False

def parse_data(data: dict) -> dict:

  date = datetime.now().strftime('%Y-%m-%d')
  timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

  parsed_data = dict()

  for key, value in data.items():

      if key == 'from':
          for k, v in data[key].items():
              if k in ['id', 'is_bot', 'first_name']:
                parsed_data[f"{key if key == 'chat' else 'user'}_{k}"] = [v]

      elif key == 'chat':
          for k, v in data[key].items():
              if k in ['id', 'type']:
                parsed_data[f"{key if key == 'chat' else 'user'}_{k}"] = [v]

      elif key in ['message_id', 'date', 'text']:
          parsed_data[key] = [value]

  if not 'text' in parsed_data.keys():
    parsed_data['text'] = [None]

  return parsed_data
```

O scripts retorna a um *bucket*, uma tabela no formato *parquet* organizada com os seguintes dados:



* Telegram Schema

| Column Name      | Data Type |
|------------------|-----------|
| message_id       | int64     |
| user_id          | int64     |
| user_is_bot      | bool      |
| user_first_name  | object    |
| chat_id          | int64     |
| chat_type        | object    |
| date             | int64     |
| text             | object    |

### 3.4 Carregamento

#### 3.4 AWS Event Bridge
É um serviço que permite a criação de regras, muitas vezes baseadas em cronogramas, para acionar eventos específicos, facilitando a automação de tarefas recorrentes. Essa funcionalidade é útil para programar a execução de determinadas ações, como a ativação das funções Lambda deste projeto, conforme a agenda predefinida.



O *AWS Event Bridge* foi responsável por inicializar o processo de transformação, uma vez ao dia às 05:00h. O scripts Python de transformação é executado.

![Eventos](https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/agendamento%20das%20execu%C3%A7%C3%B5es.png?raw=true)



#### 3.4 AWS S3
Após a execução automazida os aquivos já possivelmente foram salvos e armazenados em um novo `BUCKET`, já com os dados prontos para consumo de geração de insights.



<img src="https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/arquivos%20salvos%20em%20parquet.png?raw=truee">

<a id='apres'></a>
## 4. Apresentação
Nesta fase os dados são disponibilizados para os usuários finais. Geralmente, as informações são acessadas por meio de ferramentas de consulta, como *SQL*, sendo esta a principal interface para a maioria dos usuários. Nesse contexto, a etapa de apresentação utiliza o *AWS Athena*, uma ferramenta com motor de consulta *SQL*, simplificando a leitura e visualização dos dados armazenados na camada *ETL* para análises eficazes.

### 4.1 AWS Athena
O *AWS Athena* é um serviço de consulta interativa que permite analisar dados armazenados no Amazon S3 usando SQL padrão. Ele elimina a necessidade de carregar dados para um banco de dados permitindo explorar grandes conjuntos de dados de maneira fácil e flexível, obtendo insights valiosos sem a necessidade de infraestrutura prévia ou complexos processos de gerenciamento de dados. É especialmente útil em cenários de *Big Data* e *Data Lakes*, como no nosso caso, proporcionando uma abordagem ágil para análise de dados na nuvem. É o meio escolhido para este projeto, para visualizar e analisar as informações armazenadas.



Na etapa de **apresentação**, o *AWS Athena* tem função de entregar o dados através de uma interface apropriada para os usuários do sistema analítico. Por tanto foi criado, uma "tabela", com nome de `telegram`, para  analisados brevemente as informações obtidas. Para a criação das tabelas foi usado a linguagem *SQL*:



Criação da tabela SQL - Telegram

```sql
CREATE EXTERNAL TABLE `telegram`(
  `message_id` bigint,
  `user_id` bigint,
  `user_is_bot` boolean,
  `user_first_name` string,
  `chat_id` bigint,
  `chat_type` string,
  `text` string,
  `date` bigint)
PARTITIONED BY (
  `context_date` date)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://<bucket>/telegram/'
```

Após criação da tabela é sempre importante executar o comando abaixo, para informar ao Athena que existem novos arquivos no Bucket, isso irá garantir que os dados sejam lidos e consultados por ele.

```sql
MSCK REPAIR TABLE `telegram`
```

A consulta SQL a seguir foi projetada para validar a criação bem-sucedida de uma das tabelas em nosso banco de dados. Além disso, ela confirma se os dados particionados foram corretamente capturados, armazenados e depois recuperados no serviço S3. Esta verificação é crucial para garantir a integridade dos dados antes de prosseguirmos com qualquer análise adicional.

```sql
SELECT * FROM "telegram";
```
![Resposta SQL](https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/sql1.png?raw=true)

### 4.2 Análise Exploratória de Dados
Nesta seção, aprofundamos a análise dos dados enriquecidos por meio de consultas, buscando entender melhor as nuances e informações presentes. As consultas SQL feitas no serviço *Athena* nos possibilita a criar análises exploratórias e extração de insights, com essas conclusões podemos entender melhor o comportamento dos usuários do grupo e, a partir disso, criar estratégias para melhorar a experiência deles.

A seguir, uma consulta SQL realizada para uma análise dos dados enriquecidos da tabela `telegram`. Inicialmente, converte o campo de data/hora (`timestamp`) para um formato legível de carimbo de data/hora. Posteriormente, extrai informações como a hora do dia, o dia da semana e o número da semana a partir do carimbo de data/hora. Por fim, agrupa os dados, contabilizando o número de mensagens para cada combinação de hora, dia da semana e número da semana. Isso proporciona um resumo da frequência das mensagens, destacando padrões temporais relevantes.

```sql
WITH
parsed_date_cte AS (
    SELECT
        *,
        CAST(date_format(from_unixtime("date"),'%Y-%m-%d %H:%i:%s') AS timestamp) AS parsed_date
    FROM "whatsapp"
),
hour_week_cte AS (
    SELECT
        *,
        EXTRACT(hour FROM parsed_date) AS hora_do_dia,
        EXTRACT(dow FROM parsed_date) AS dia_da_semana,
        EXTRACT(week FROM parsed_date) AS semana_do_ano
    FROM parsed_date_cte
)
SELECT
    hora_do_dia,
    dia_da_semana,
    semana_do_ano,
    count(1) AS "qtd_de_mensagens"
FROM hour_week_cte
GROUP BY
    hora_do_dia,
    dia_da_semana,
    semana_do_ano
ORDER BY
    semana_do_ano,
    dia_da_semana
```
![Resposta SQL](https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/sql2.png?raw=true)

Essa consulta SQL está sendo usada para obter a quantidade de mensagens enviadas por cada usuário no aplicativo “telegram” em cada data específica. Aqui está o que cada parte da consulta faz:

`SELECT user_id, user_first_name`, `context_date, count(1) AS "message_amount"`: Esta parte seleciona as colunas de interesse. A função count(1) conta o número de linhas para cada combinação única de user_id, user_first_name e context_date, e o resultado é rotulado como message_amount.
FROM "telegram": Esta parte indica que os dados estão sendo retirados da tabela “telegram”.

`GROUP BY user_id, user_first_name, context_date`: Esta parte agrupa os dados por user_id, user_first_name e context_date. Isso significa que para cada combinação única dessas três variáveis, a consulta contará o número de linhas (ou seja, o número de mensagens).

`ORDER BY context_date DESC`: Esta parte ordena os resultados pela coluna context_date em ordem decrescente, o que significa que as datas mais recentes aparecerão primeiro.
Portanto, o propósito desta consulta é fornecer uma visão de quantas mensagens cada usuário enviou em cada data, com os resultados ordenados do mais recente ao mais antigo. Isso pode ser útil para entender o comportamento do usuário ao longo do tempo. Por exemplo, você pode ver se um usuário está se tornando mais ativo ou menos ativo no aplicativo.

```sql
SELECT
  user_id,
  user_first_name,
  context_date,
  count(1) AS "message_amount"
FROM "telegram"
GROUP BY
  user_id,
  user_first_name,
  context_date
ORDER BY context_date DESC
````
![Resposta SQL](https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/sql3.png?raw=true)

O propósito desta consulta é fornecer uma visão do comprimento médio das mensagens que cada usuário enviou em cada data, com os resultados ordenados do mais recente ao mais antigo. Isso pode ser útil para entender o comportamento do usuário ao longo do tempo. Por exemplo, você pode ver se um usuário está enviando mensagens mais longas ou mais curtas ao longo do tempo.

```sql
SELECT
  user_id,
  user_first_name,
  context_date,
  CAST(AVG(length(text)) AS INT) AS "average_message_length"
FROM "telegram"
GROUP BY
  user_id,
  user_first_name,
  context_date
ORDER BY context_date DESC
````
![Resposta SQL](https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/sql4.png?raw=true)

O propósito desta consulta é fornecer uma visão de quantas mensagens foram enviadas no aplicativo “telegram” em cada data, com os resultados ordenados do mais recente ao mais antigo. Isso pode ser útil para entender o volume de mensagens ao longo do tempo. Por exemplo, você pode ver se o volume de mensagens está aumentando ou diminuindo ao longo do tempo.

```sql
SELECT
  context_date,
  count(1) AS "message_amount"
FROM "telegram"
GROUP BY context_date
ORDER BY context_date DESC
````
![Resposta SQL](https://github.com/edersonss1987/AutomacaoTelegram/blob/main/imagens/sql5.png?raw=true)

<a id='conclu'></a>
## 5. Conclusão

A integração do Telegram e a análise de dados com serviços de nuvem, como a AWS, melhoram significativamente as operações de uma empresa. Essas tecnologias permitem que os clientes interajam facilmente com a empresa a qualquer hora, pois estão sempre disponíveis. A AWS ajuda a tornar essa interação ainda melhor, pois permite que a empresa ajuste seus recursos de TI rapidamente, conforme necessário, garantindo que o serviço seja escalável, flexível e confiável.

Com a análise de dados realizada na nuvem, as empresas podem processar grandes quantidades de informações de forma eficiente e segura. Isso ajuda a entender melhor os comportamentos e preferências dos clientes, analisando os dados coletados durante as interações com eles.

Além disso, usar a nuvem pode reduzir custos, pois não é necessário investir em equipamentos físicos. Também permite que as empresas se adaptem rapidamente a mudanças nas necessidades do mercado, mantendo a eficiência. A segurança é uma prioridade, e serviços como a AWS têm medidas fortes para proteger os dados dos clientes contra ataques online.

Em resumo, essa combinação de tecnologias melhora a forma como os clientes usam os aplicativos de mensagem, aumenta a eficiência das operações da empresa e ajuda na tomada de decisões estratégicas. A análise de dados é essencial para as empresas hoje em dia, pois fornece as ferramentas necessárias para inovar e alcançar a excelência no mercado.